# Advanced Pandas Interview — Hands‑On Practice (with Solutions)

**Goal:** Practice advanced, interview‑style Pandas problems that emphasize *correctness*, *expressiveness*, and *performance*. Most tasks create small synthetic datasets so you can run everything locally.

> Tip: Try to complete the **Your turn** cells before revealing/reading the **Solution** cells right below them.

**Covers:** GroupBy, Window/Rolling/EWM, Reshaping & MultiIndex, Joins (incl. as‑of & interval), Time series, Missing data & conditionals, Text & explode, Categoricals, Index alignment & broadcasting, Method chaining & `pipe`, and a Cohort bonus.

Tested with **pandas ≥ 1.5** and **Python ≥ 3.9**.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

pd.set_option('display.width', 120)
pd.set_option('display.max_columns', 50)
np.random.seed(42)

print(pd.__version__)

ModuleNotFoundError: No module named 'numpy'

## 1) GroupBy Mastery
We'll build a synthetic dataset with groups `A`, `B` and numeric columns `X`, `Y`.

In [ ]:
# --- Data: GroupBy ---
N = 500
A = np.random.choice(list('ABC'), size=N, p=[0.4, 0.4, 0.2])
B = np.random.choice(['u','v','w'], size=N)
X = np.random.normal(loc=100, scale=20, size=N)
Y = np.random.lognormal(mean=4.0, sigma=0.5, size=N)

df = pd.DataFrame({'A': A, 'B': B, 'X': X, 'Y': Y})
df.head()

### 1.1 Mixed aggregations with named outputs
**Your turn:** Aggregate by `['A','B']` computing:
- `mean_x = mean(X)`
- `n = size`
- `p90_y = 90th percentile of Y`

Return a tidy DataFrame with a simple index.

In [ ]:
# TODO: your solution here
# out = (
#     df.groupby(['A', 'B'])
#       .agg(
#           mean_x=(..., ...),
#           n=(..., ...),
#           p90_y=(..., ...),
#       )
#       .reset_index()
# )
# out.head()

**Solution:**

In [ ]:
out = (
    df.groupby(['A', 'B'])
      .agg(
          mean_x=('X', 'mean'),
          n=('X', 'size'),
          p90_y=('Y', lambda s: s.quantile(0.90))
      )
      .reset_index()
)

# simple checks
assert {'A','B','mean_x','n','p90_y'} <= set(out.columns)
assert out.shape[0] <= df[['A','B']].drop_duplicates().shape[0]
out.sort_values(['A','B']).head()

### 1.2 `agg` vs `transform` — groupwise z‑score
**Your turn:** Create `zscore_X` = z‑score of `X` within each `A` group: `(X - mean_A) / std_A` using **`transform`**.

In [ ]:
# TODO: your solution here
# df['zscore_X'] = df.groupby('A')['X'].transform(lambda s: ...)
# df[['A','X','zscore_X']].head()

**Solution:**

In [ ]:
df['zscore_X'] = df.groupby('A')['X'].transform(lambda s: (s - s.mean()) / s.std(ddof=0))
assert np.isclose(df.groupby('A')['zscore_X'].mean().abs().max(), 0, atol=1e-6)
df[['A','X','zscore_X']].head()

### 1.3 Filter groups efficiently
**Your turn:** Keep only groups with at least **30** rows (by `A,B`).

In [ ]:
# TODO: your solution here
# keep = df.groupby(['A','B'])['X'].transform('size') >= 30
# df_big = df[keep]
# df_big['AB'] = df_big['A'] + df_big['B']
# df_big.head()

**Solution:**

In [ ]:
keep = df.groupby(['A','B'])['X'].transform('size') >= 30
df_big = df[keep].copy()
assert df_big.groupby(['A','B']).size().min() >= 30
len(df), len(df_big)

## 2) Window / Rolling / Expanding / EWM
Create per‑ID time series and compute rolling/exp-weighted features.

In [ ]:
# --- Data: Time series ---
ids = np.repeat(np.arange(5), 60)
base = pd.Timestamp('2023-01-01')
ts = np.concatenate([pd.date_range(base, periods=60, freq='D').values for _ in range(5)])
val = np.random.normal(0, 1, size=len(ids)).cumsum() + np.repeat(np.random.randn(5)*5, 60)

ts_df = pd.DataFrame({'id': ids, 'ts': pd.to_datetime(ts), 'value': val}).sort_values(['id','ts'])
ts_df.head()

### 2.1 Time‑based rolling window
**Your turn:** For each `id`, compute a 30‑day rolling **sum** of `value` with `min_periods=1` using a **time‑based** window (i.e., `rolling('30D')`).

In [ ]:
# TODO: your solution here
# roll_sum = (ts_df.set_index('ts')
#                 .groupby('id')['value']
#                 .rolling('30D', min_periods=1)
#                 .sum()
#                 .reset_index(level=0, drop=True))
# ts_df['roll30d_sum'] = roll_sum
# ts_df.head()

**Solution:**

In [ ]:
roll_sum = (ts_df.set_index('ts')
                .groupby('id')['value']
                .rolling('30D', min_periods=1)
                .sum()
                .reset_index(level=0, drop=True))

ts_df['roll30d_sum'] = roll_sum
assert ts_df['roll30d_sum'].notna().all()
ts_df.head()

### 2.2 Exponentially‑weighted mean per `id`
**Your turn:** Compute `ewm_mean` with `span=10` per `id`.

In [ ]:
# TODO: your solution here
# ts_df['ewm_mean'] = ts_df.groupby('id')['value'].transform(lambda s: s.ewm(span=10).mean())
# ts_df.head()

**Solution:**

In [ ]:
ts_df['ewm_mean'] = ts_df.groupby('id')['value'].transform(lambda s: s.ewm(span=10).mean())
assert ts_df['ewm_mean'].isna().sum() == 0

### 2.3 Custom rolling statistic
**Your turn:** Compute a 7‑point rolling **range** (`max - min`) per `id` with `min_periods=3`.

In [ ]:
# TODO: your solution here
# r = (ts_df.groupby('id')['value']
#           .rolling(7, min_periods=3)
#           .agg(lambda x: x.max() - x.min())
#           .reset_index(level=0, drop=True))
# ts_df['roll7_range'] = r
# ts_df.head()

**Solution:**

In [ ]:
r = (ts_df.groupby('id')['value']
          .rolling(7, min_periods=3)
          .agg(lambda x: x.max() - x.min())
          .reset_index(level=0, drop=True))

ts_df['roll7_range'] = r
assert ts_df['roll7_range'].isna().sum() > 0  # first few rows per id are NaN

## 3) Reshaping & MultiIndex: `pivot`, `pivot_table`, `melt`, `stack`, `unstack`

In [ ]:
# --- Data: transactions ---
_dates = pd.date_range('2023-01-01', periods=30, freq='D')
_cats = ['electronics','books','toys']
rows = []
for d in _dates:
    for c in _cats:
        sales = np.random.poisson(lam={'electronics':15,'books':25,'toys':10}[c])
        qty = sales // np.random.randint(1,4)
        rows.append((d, c, sales, qty))
trans = pd.DataFrame(rows, columns=['date','category','sales','qty'])
trans.head()

### 3.1 Pivot table with multiple values and margins
**Your turn:** Create a wide table with index=`date`, columns=`category`, values=`['sales','qty']`, aggregating by **sum**, `fill_value=0`, and include totals (`margins=True`).

In [ ]:
# TODO: your solution here
# w = trans.pivot_table(index='date', columns='category', values=['sales','qty'], aggfunc='sum', fill_value=0, margins=True)
# w.head()

**Solution:**

In [ ]:
w = trans.pivot_table(index='date', columns='category', values=['sales','qty'], aggfunc='sum', fill_value=0, margins=True)
assert isinstance(w.columns, pd.MultiIndex)
w.head()

### 3.2 Long ↔ Wide with `stack` / `melt`
**Your turn:** Convert `w` back to long form with columns `date`, `category`, `metric` (`sales` or `qty`), and `value`.

In [ ]:
# TODO: your solution here
# long = (w.drop(index='All')
#          .stack(level=1)
#          .rename_axis(['date','category'])
#          .reset_index())
# long = long.rename(columns={0:'value'})
# long['metric'] = long.columns.get_level_values(0) if isinstance(long.columns, pd.MultiIndex) else 'value'
# long.head()

**Solution:**

In [ ]:
# Drop the grand total row to keep tidy semantics
w_no_all = w.drop(index='All')
long = (
    w_no_all.stack(level=1)  # stack categories
          .rename_axis(['date','category'])
          .reset_index()
)
# After stacking, columns are still MultiIndex (level 0: metric)
long = long.rename(columns={'sales':'sales', 'qty':'qty'})
long = long.melt(id_vars=['date','category'], value_vars=['sales','qty'], var_name='metric', value_name='value')
assert set(long['metric'].unique()) == {'sales','qty'}
long.head()

## 4) Joins (incl. as‑of, cross, and interval mapping)

In [ ]:
# --- Data: joins ---
users = pd.DataFrame({
    'user_id': [1,2,3,4],
    'tier': ['silver','gold','gold','platinum']
})
orders = pd.DataFrame({
    'order_id':[10,11,12,13,14],
    'user_id':[1,2,2,3,4],
    'amount':[120.0, 55.0, 75.0, 200.0, 500.0]
})
users, orders

### 4.1 Basic merge with integrity checks
**Your turn:** Left‑join `orders` with `users` on `user_id`, include `_merge` indicator, and validate `m:1`.

In [ ]:
# TODO: your solution here
# merged = pd.merge(orders, users, on='user_id', how='left', indicator=True, validate='m:1')
# merged

**Solution:**

In [ ]:
merged = pd.merge(orders, users, on='user_id', how='left', indicator=True, validate='m:1')
assert set(merged['_merge'].unique()) == {'both'}
merged

### 4.2 As‑of join (nearest by time per key)
**Your turn:** For each trade, attach the latest quote up to that time (per symbol) within 2 minutes.

In [ ]:
# Data: trades & quotes
quotes = pd.DataFrame({
    'symbol':['A']*5 + ['B']*5,
    'time':  list(pd.date_range('2023-01-01 09:30', periods=5, freq='T')) +
             list(pd.date_range('2023-01-01 09:30', periods=5, freq='2T')),
    'bid':[99,99.5,100,100.5,101, 50,50.5,51,51.5,52],
})
trades = pd.DataFrame({
    'symbol':['A','A','B','B'],
    'time':  [pd.Timestamp('2023-01-01 09:31:20'), pd.Timestamp('2023-01-01 09:33:00'),
              pd.Timestamp('2023-01-01 09:31:00'), pd.Timestamp('2023-01-01 09:35:00')],
    'qty':[100,200,150,300],
})

quotes = quotes.sort_values(['symbol','time'])
trades = trades.sort_values(['symbol','time'])

# TODO: your solution here
# matched = pd.merge_asof(trades, quotes, on='time', by='symbol', direction='backward', tolerance=pd.Timedelta('2min'))
# matched

**Solution:**

In [ ]:
matched = pd.merge_asof(trades, quotes, on='time', by='symbol', direction='backward', tolerance=pd.Timedelta('2min'))
assert matched['bid'].notna().all()
matched

### 4.3 Cross join (Cartesian product)
**Your turn:** Create all combinations of `users` and a small `scenarios` table.

In [ ]:
scenarios = pd.DataFrame({'scenario':['base','stress']})
# TODO: your solution here
# cx = users.merge(scenarios, how='cross')
# cx.head()

**Solution:**

In [ ]:
cx = users.merge(scenarios, how='cross')
assert cx.shape[0] == len(users) * len(scenarios)
cx.head()

### 4.4 Interval mapping (non‑equi join)
**Your turn:** Map each value to a `[low, high]` band.

In [ ]:
# Data: bands and values
bands = pd.DataFrame({'band':['L','M','H'], 'low':[0, 100, 200], 'high':[99, 199, 1_000]})
values = pd.DataFrame({'value':[10, 150, 450, 220, 90]})

# TODO: your solution here — use IntervalIndex and merge
# intervals = pd.IntervalIndex.from_arrays(bands['low'], bands['high'], closed='both')
# bands2 = bands.set_index(pd.Index(intervals, name='interval'))
# out = pd.merge(values, bands2.reset_index(), left_on='value', right_on='interval', how='left')
# out

**Solution:**

In [ ]:
intervals = pd.IntervalIndex.from_arrays(bands['low'], bands['high'], closed='both')
bands2 = bands.set_index(pd.Index(intervals, name='interval'))
out_intervals = pd.merge(values, bands2.reset_index(), left_on='value', right_on='interval', how='left')
assert out_intervals['band'].notna().sum() == len(values)
out_intervals

## 5) Time Series Essentials

In [ ]:
# --- Data: irregular events ---
raw = pd.DataFrame({
    'ts': pd.to_datetime(['2023-01-01 09:00','2023-01-02 13:30','2023-01-05 10:00']),
    'value':[10, np.nan, 25]
})
raw

### 5.1 Resample to business days with forward fill (limit=2)
**Your turn:** Set `ts` as index, resample to 'B', forward‑fill with a maximum gap of 2 steps.

In [ ]:
# TODO: your solution here
# out = (raw.set_index('ts')
#            .resample('B')
#            .ffill(limit=2)
#            .reset_index())
# out.head(10)

**Solution:**

In [ ]:
out_ts = (raw.set_index('ts')
             .resample('B')
             .ffill(limit=2)
             .reset_index())
assert out_ts['value'].isna().sum() >= 0
out_ts.head(10)

### 5.2 Timezones — normalize to UTC then convert
**Your turn:** Treat the times as US/Eastern, convert to UTC.

In [ ]:
# TODO: your solution here
# tz = pd.Series(pd.to_datetime(raw['ts'])).dt.tz_localize('US/Eastern').dt.tz_convert('UTC')
# tz.head()

**Solution:**

In [ ]:
tz = pd.Series(pd.to_datetime(raw['ts'])).dt.tz_localize('US/Eastern').dt.tz_convert('UTC')
assert str(tz.dt.tz.iloc[0]) == 'UTC'
tz.head()

## 6) Missing Data & Conditional Logic

In [ ]:
miss = pd.DataFrame({'x':[1, -1, 5, -3, 2], 'y':[np.nan, 3.0, np.nan, 4.0, 5.0], 'auto':[None, 'a', None, 'b', None], 'manual':['A', None, None, None, 'E']})
miss

### 6.1 NA‑safe dtypes and `where`
**Your turn:** Create `adj` = `x` if `x ≥ 0` else `NA`, using nullable `Int64` dtype.

In [ ]:
# TODO: your solution here
# miss['adj'] = miss['x'].where(miss['x'] >= 0, other=pd.NA).astype('Int64')
# miss

**Solution:**

In [ ]:
miss['adj'] = miss['x'].where(miss['x'] >= 0, other=pd.NA).astype('Int64')
assert str(miss['adj'].dtype) == 'Int64'
miss

### 6.2 Coalesce with `combine_first`
**Your turn:** Create `best` that prefers `manual` over `auto`.

In [ ]:
# TODO: your solution here
# miss['best'] = pd.Series(miss['manual'], dtype='string').combine_first(pd.Series(miss['auto'], dtype='string'))
# miss

**Solution:**

In [ ]:
miss['best'] = pd.Series(miss['manual'], dtype='string').combine_first(pd.Series(miss['auto'], dtype='string'))
assert miss['best'].isna().sum() == 0
miss

### 6.3 Groupwise interpolate by time
**Your turn:** Interpolate missing `value` per `id` using `method='time'`.

In [ ]:
# Data: groupwise interpolation
interp = pd.DataFrame({
    'id': np.repeat([1,2], 5),
    'ts': list(pd.date_range('2023-01-01', periods=5, freq='D'))*2,
    'value':[1, np.nan, 3, np.nan, 5, 10, np.nan, 14, np.nan, 18]
})

# TODO: your solution here
# out = (interp.sort_values(['id','ts'])
#             .set_index('ts')
#             .groupby('id')['value']
#             .apply(lambda s: s.interpolate('time'))
#             .reset_index(level=0, drop=True)
#             .reset_index())
# out.head()

**Solution:**

In [ ]:
out_interp = (interp.sort_values(['id','ts'])
                .set_index('ts')
                .groupby('id')['value']
                .apply(lambda s: s.interpolate('time'))
                .reset_index(level=0, drop=True)
                .reset_index())
assert out_interp['value'].isna().sum() == 0
out_interp.head()

## 7) Vectorized Text & `explode`

In [ ]:
text_df = pd.DataFrame({
    'user':['u1','u2','u3'],
    'text':['Area: 120; Height: 50', 'Area: 90; Height: 60', 'Height: 70'],
    'tags':['red;green', 'blue', 'green;yellow']
})
text_df

### 7.1 Regex extract with named group
**Your turn:** Extract `area` as integer from `text` (missing if absent).

In [ ]:
# TODO: your solution here
# pat = r"Area:\s*(?P<area>\d+)"
# text_df['area'] = text_df['text'].str.extract(pat, expand=False).astype('Int64')
# text_df

**Solution:**

In [ ]:
pat = r"Area:\s*(?P<area>\d+)"
text_df['area'] = text_df['text'].str.extract(pat, expand=False).astype('Int64')
assert text_df['area'].dtype == 'Int64'
text_df

### 7.2 Split to rows with `explode` and aggregate
**Your turn:** Split `tags` on `;`, explode to rows, and compute number of unique tags per user.

In [ ]:
# TODO: your solution here
# exploded = text_df.assign(tag=text_df['tags'].str.split(';')).explode('tag')
# out = exploded.groupby('user')['tag'].nunique().reset_index(name='n_tags')
# out

**Solution:**

In [ ]:
exploded = text_df.assign(tag=text_df['tags'].str.split(';')).explode('tag')
out_tags = exploded.groupby('user')['tag'].nunique().reset_index(name='n_tags')
assert out_tags['n_tags'].min() >= 1
out_tags

## 8) Categoricals & custom sort order

In [ ]:
cat_df = pd.DataFrame({'tier':['gold','platinum','silver','gold','silver'], 'revenue':[200, 1000, 120, 350, 180]})
cat_df

### 8.1 Sort tiers by custom order
**Your turn:** Use ordered `CategoricalDtype(['silver','gold','platinum'])` then sort by `tier` asc, `revenue` desc.

In [ ]:
# TODO: your solution here
# order = pd.CategoricalDtype(['silver','gold','platinum'], ordered=True)
# cat_df['tier'] = cat_df['tier'].astype(order)
# out = cat_df.sort_values(['tier','revenue'], ascending=[True, False])
# out

**Solution:**

In [ ]:
order = pd.CategoricalDtype(['silver','gold','platinum'], ordered=True)
cat_df['tier'] = cat_df['tier'].astype(order)
out_cat = cat_df.sort_values(['tier','revenue'], ascending=[True, False]).reset_index(drop=True)
assert list(out_cat['tier'].astype(str).unique()) == ['silver','gold','platinum']
out_cat

## 9) Index alignment, broadcasting, and smart sorting

In [ ]:
align_df = pd.DataFrame({'grp': list('AAABBB'), 'x':[5,7,9, 2,4,6]})
align_df

### 9.1 Center `x` by group mean (alignment)
**Your turn:** Create `x_centered = x - mean(x) per grp`.

In [ ]:
# TODO: your solution here
# m = align_df.groupby('grp')['x'].transform('mean')
# align_df['x_centered'] = align_df['x'] - m
# align_df

**Solution:**

In [ ]:
m = align_df.groupby('grp')['x'].transform('mean')
align_df['x_centered'] = align_df['x'] - m
assert np.isclose(align_df.groupby('grp')['x_centered'].mean().abs().max(), 0)
align_df

### 9.2 Vectorized conditionals with `map` / `replace`
**Your turn:** Map `grp` A→'alpha', B→'beta' into `label`.

In [ ]:
# TODO: your solution here
# align_df['label'] = align_df['grp'].map({'A':'alpha','B':'beta'})
# align_df

**Solution:**

In [ ]:
align_df['label'] = align_df['grp'].map({'A':'alpha','B':'beta'})
assert set(align_df['label'].unique()) == {'alpha','beta'}
align_df

### 9.3 Sort with a key function
**Your turn:** Sort strings by length using `key=`.

In [ ]:
# Data
srt = pd.DataFrame({'s':['bbb','a','ccccc','dddd']})
# TODO: your solution here
# srt_sorted = srt.sort_values('s', key=lambda s: s.str.len())
# srt_sorted

**Solution:**

In [ ]:
srt = pd.DataFrame({'s':['bbb','a','ccccc','dddd']})
srt_sorted = srt.sort_values('s', key=lambda s: s.str.len()).reset_index(drop=True)
assert list(srt_sorted['s']) == ['a','bbb','dddd','ccccc']
srt_sorted

## 10) Method chaining & `pipe`

In [ ]:
pipe_df = pd.DataFrame({
    'account':['x','x','y','y','y'],
    'status':['open','closed','open','open','closed'],
    'qty':[10,5,2,8,4],
    'price':[100,110,200,50,80]
})
pipe_df

### 10.1 Build a clean pipeline
**Your turn:**
1. Filter `status == "open"`
2. Add `cost = qty * price`
3. Keep only rows with `cost > 300`
4. Sort by `account` asc, `cost` desc

In [ ]:
# TODO: your solution here
# def keep_costly(d, th=300):
#     return d[d['cost'] > th]
# out = (
#     pipe_df.query('status == "open"')
#            .assign(cost=lambda d: d['qty'] * d['price'])
#            .pipe(keep_costly, th=300)
#            .sort_values(['account','cost'], ascending=[True, False])
# )
# out

**Solution:**

In [ ]:
def keep_costly(d, th=300):
    return d[d['cost'] > th]

out_pipe = (
    pipe_df.query('status == "open"')
           .assign(cost=lambda d: d['qty'] * d['price'])
           .pipe(keep_costly, th=300)
           .sort_values(['account','cost'], ascending=[True, False])
)
assert (out_pipe['cost'] > 300).all()
out_pipe

## Bonus) Cohort Analysis (first purchase month)
**Your turn:** Compute each user's first purchase month and add a `cohort_index` (months since first purchase).

In [ ]:
# Data
co = pd.DataFrame({
    'user': np.repeat(['u1','u2','u3'], [3,4,2]),
    'ts':   pd.to_datetime(['2023-01-15','2023-03-10','2023-03-25',
                            '2023-02-02','2023-02-15','2023-04-01','2023-04-20',
                            '2023-05-05','2023-07-10']),
    'amount':[100,50,75, 40,60,80,90, 120, 200]
})
co

In [ ]:
# TODO: your solution here
# co['month'] = co['ts'].dt.to_period('M')
# first = co.groupby('user')['month'].min().rename('cohort')
# co = co.join(first, on='user')
# co['cohort_index'] = (co['month'] - co['cohort']).apply(lambda p: p.n)
# co.sort_values(['user','ts'])

**Solution:**

In [ ]:
co['month'] = co['ts'].dt.to_period('M')
first = co.groupby('user')['month'].min().rename('cohort')
co = co.join(first, on='user')
co['cohort_index'] = (co['month'] - co['cohort']).apply(lambda p: p.n)
assert co.groupby('user')['cohort_index'].min().eq(0).all()
co.sort_values(['user','ts'])

---
### Tips for Interviews
- Prefer `agg`/`transform` to `apply` where possible; vectorize.
- Use `validate=` in merges to catch integrity issues early.
- For time series, always `sort_values` and standardize timezones.
- Use nullable dtypes (`Int64`, `boolean`, `string`) for NA‑safe operations.
- Consider categoricals for memory/speed and correct custom ordering.

Good luck! 🚀